<a href="https://colab.research.google.com/github/webb-e/S2_Landsat_Comparison/blob/main/WeeklyMosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project=" ") # Initialize the library.

import itertools

In [ ]:
regionfolder = None
region_name = None
roi = None
lakes = None
def regionfun(region):
  global regionfolder, region_name, roi, lakes
  if region == 'TUK/MRD':
    regionfolder = 'projects/alpod-412314/assets/TUK_MRD_weekly/'
    region_name = 'TUK_MRD'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/MRD_TUK_Anderson_aoi")
    lakes = ee.FeatureCollection("projects/alpod-412314/assets/just_lakes/justlakes_TUKMRDAND")
  elif region == 'AKCP':
    regionfolder = 'projects/alpod-412314/assets/AKCP_weekly/'
    region_name = 'AKCP'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/coastalplain_roi")
    lakes = ee.FeatureCollection("projects/alpod-412314/assets/just_lakes/justlakes_AKCP")
  elif region == 'YKD':
    regionfolder = 'projects/alpod-412314/assets/YKD_weekly/'
    region_name = 'YKD'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/YKdelta_roi")
    lakes = ee.FeatureCollection("projects/alpod-412314/assets/just_lakes/justlakes_YKD")
  elif region == 'YKF':
    regionfolder = 'projects/alpod-412314/assets/YKF_weekly/'
    region_name = 'YKF'
    roi = ee.FeatureCollection("projects/ee-webbe/assets/ROIs/YKflats_roi")
    lakes = ee.FeatureCollection("projects/alpod-412314/assets/just_lakes/justlakes_YKF")
  else:
    print("Invalid region")


In [ ]:
regionfun('TUK/MRD') # options = TUK/MRD, AKCP, YKD, YKF

## define function to remove bad tiles (i.e, tiles where water occurence is <50% of expected)
def rmbadtiles(tile):
    #####
    #### FIRST: determine expected area of water and number of lakes
    #####

    ##Create a binary mask where the unmasked pixels are 1 and everything else is 0.
    binaryMask = tile.mask().reduce(ee.Reducer.max());
    ##sum unmaksed pixels within each lake
    possiblewater = binaryMask.reduceRegions(
          collection= lakes,
          reducer= ee.Reducer.sum(),
          scale= 10 )
    ## filter out lakes that are masked (i.e., sum = 0)
    lakesintile = possiblewater.filter(ee.Filter.gt('sum', 0))
    ## expected area of water within each unmasked lake
    expectedwater = lakesintile.aggregate_sum('area_km2')
    ## possible number of lakes
  #  possiblelakes = ee.FeatureCollection(lakesintile).size()

    #####
    #### SECOND: determine observed area of water and number of lakes
    #####
    ## Replace masked values with 1
    not_masked_img = tile.unmask(1);
    ## Get only values with 1 and clip to unmasked lakes
    waterinimage = not_masked_img.updateMask(not_masked_img.eq(1))
    areawaterinimage = ee.Image.pixelArea().multiply(waterinimage).clipToCollection(lakesintile)
    ## get the observed area of water pixels
    obswater = areawaterinimage.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= tile.geometry(),
        scale= 10,
        maxPixels=1e12)
    ##convert to km2
    observedwater = ee.Number(obswater.get('area')).divide(1e6);
    ## get the observed number of water pixels within unmasked lakes
    observedlakewater = waterinimage.reduceRegions(
          collection= lakesintile,
          reducer= ee.Reducer.sum(),
          scale= 10 )
    ## get number of lakes
   # observedlakes = ee.FeatureCollection(observedlakewater.filter(ee.Filter.gt('sum', 0))).size()

  #####
  #### THIRD: calculate the ratio of observed:expected
  #####
    area_ratio = observedwater.divide(expectedwater)
   # lake_ratio = ee.Number(observedlakes).divide(ee.Number(possiblelakes))
    ### if the image has <50% of expected water or <30% of expected lakes, delete image
    cond1 = area_ratio.lte(0.5)
   # cond2 = lake_ratio.lte(0.3)
   # combinedCondition = cond1.Or(cond2)
    return ee.Algorithms.If(cond1, tile.set('isNull', True), tile.set('isNull', False))

### get list of all occurance images in the  region and filter for each week of each year
assetList = ee.data.listAssets(regionfolder)['assets']
imgs = ee.ImageCollection.fromImages([ee.Image(asset['id']) for asset in assetList])
years = list(range(2016,2024))
weeks = list(range(18,41))
aoi = ee.Geometry.MultiPolygon(roi.geometry().getInfo()['coordinates'])
for year, week in itertools.product(years, weeks):
    year = str(year)
    week = str(week)
    filteredimgs = imgs.filter(ee.Filter.stringContains('system:id',year)).filter(ee.Filter.stringEndsWith('system:id',week))
    ## remove bad images and take the max value
    goodimgs = filteredimgs.map(rmbadtiles).filter(ee.Filter.eq('isNull', False))
    max_image = goodimgs.reduce(ee.Reducer.max())
    description = region_name + '_' + year + '_' + week
    task = ee.batch.Export.image.toAsset(**{
              'image': ee.Image(max_image),
              'description': description,
              'assetId': 'projects/alpod-412314/assets/region_weekly/' + description,
              'scale': 10,
              'region': aoi,
              'maxPixels': 1e12,
              })
    task.start()
    print("Image export " +  description +  " has started")
